In [1]:
import requests #Connect to endpoints
import time #time script
import sys
import csv
import pandas as pd
import spotipy #authentication
import spotipy.util as util #authentication

from spotipy.oauth2 import SpotifyClientCredentials #authentication

In [2]:
#Credentials
cid = 'f61689bf814a44dab11d7e63d18dc38a'
secret = 'xxx'
redirect_uri = 'http://google.com/'
username = 'xxx'

In [3]:
#Authentication
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
#all features and keys should be lowercase with underscore instead of spaces
def get_audiofeatures(url,track_uri,show=True):
    features = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness',
                'liveness','valence','tempo','time_signature']
    
    #dictionary 
    audiofeatures = {}
    
    #get url
    try:
        resp = requests.get(url=url,
                            headers={'Authorization': 'Bearer '+ token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
    #response in json format
    response = resp.json()
    
    audiofeatures.update({'track_uri':track_uri})
    

    for track_feature in features:
        feature_value = response[track_feature]
        audiofeatures.update({track_feature:feature_value}) #ties track_uri and features
    
    if show:
        print('URI: {uri} - AudioFeatures: {af}'.format(uri=track_uri,af=audiofeatures))
    
    return audiofeatures          

In [5]:
i = 0
master_track_audiofeatures=[]

start = time.time()

#Open file
with open('Data/user_track_uri.csv') as csvfile:
    file = csv.reader(csvfile, delimiter='|')
    
    next(file) # Skip header
    head = [next(file) for x in range(4)] #for limiting amount of rows read
    
    for row in head:
        line = row[0].split(':') # list of items
        track_uri = line[2] #getting 1 item of the list
        print(i,line) 
        
        track_audiofeatures = get_audiofeatures('https://api.spotify.com/v1/audio-features/{id}'.format(id=track_uri),track_uri)
                                                
        #Is a list of the track_audiofeatures dictionary for each track_uri in file
        master_track_audiofeatures.append(track_audiofeatures)
        i+=1
stop = time.time()
duration = (stop - start)/60

0 ['spotify', 'track', '4bKlOuzNlpMnlnUhCvnuY8']
URI: 4bKlOuzNlpMnlnUhCvnuY8 - AudioFeatures: {'track_uri': '4bKlOuzNlpMnlnUhCvnuY8', 'danceability': 0.395, 'energy': 0.958, 'key': 1, 'loudness': -4.482, 'mode': 0, 'speechiness': 0.325, 'acousticness': 0.00928, 'instrumentalness': 0.108, 'liveness': 0.166, 'valence': 0.269, 'tempo': 74.974, 'time_signature': 4}
1 ['spotify', 'track', '6p2liQLGoDaLXgND68bfVt']
URI: 6p2liQLGoDaLXgND68bfVt - AudioFeatures: {'track_uri': '6p2liQLGoDaLXgND68bfVt', 'danceability': 0.687, 'energy': 0.677, 'key': 2, 'loudness': -4.172, 'mode': 1, 'speechiness': 0.0508, 'acousticness': 0.000658, 'instrumentalness': 0.000146, 'liveness': 0.0564, 'valence': 0.546, 'tempo': 75.052, 'time_signature': 4}
2 ['spotify', 'track', '0MsrWnxQZxPAcov7c74sSo']
URI: 0MsrWnxQZxPAcov7c74sSo - AudioFeatures: {'track_uri': '0MsrWnxQZxPAcov7c74sSo', 'danceability': 0.29, 'energy': 0.827, 'key': 8, 'loudness': -3.451, 'mode': 1, 'speechiness': 0.0362, 'acousticness': 0.000659, 'in

In [6]:
master_track_audiofeatures

[{'track_uri': '4bKlOuzNlpMnlnUhCvnuY8',
  'danceability': 0.395,
  'energy': 0.958,
  'key': 1,
  'loudness': -4.482,
  'mode': 0,
  'speechiness': 0.325,
  'acousticness': 0.00928,
  'instrumentalness': 0.108,
  'liveness': 0.166,
  'valence': 0.269,
  'tempo': 74.974,
  'time_signature': 4},
 {'track_uri': '6p2liQLGoDaLXgND68bfVt',
  'danceability': 0.687,
  'energy': 0.677,
  'key': 2,
  'loudness': -4.172,
  'mode': 1,
  'speechiness': 0.0508,
  'acousticness': 0.000658,
  'instrumentalness': 0.000146,
  'liveness': 0.0564,
  'valence': 0.546,
  'tempo': 75.052,
  'time_signature': 4},
 {'track_uri': '0MsrWnxQZxPAcov7c74sSo',
  'danceability': 0.29,
  'energy': 0.827,
  'key': 8,
  'loudness': -3.451,
  'mode': 1,
  'speechiness': 0.0362,
  'acousticness': 0.000659,
  'instrumentalness': 2.55e-06,
  'liveness': 0.301,
  'valence': 0.45,
  'tempo': 167.086,
  'time_signature': 4},
 {'track_uri': '18wCJcoFmXZ0jfrFhF6cYS',
  'danceability': 0.46,
  'energy': 0.426,
  'key': 0,
  'lou

In [7]:
#Duration of script
str(duration)+' minutes'

'0.02115054925282796 minutes'

In [8]:
df = pd.DataFrame(master_track_audiofeatures)
df
#no_duplicates = df.drop_duplicates(['Album URI']) *Change to find duplicates of track_uri
#no_duplicates
#no_duplicates.to_csv('Data/artist_album_uri.csv',sep='|',index=False)

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,4bKlOuzNlpMnlnUhCvnuY8,0.395,0.958,1,-4.482,0,0.3250,0.009280,0.108000,0.1660,0.269,74.974,4
1,6p2liQLGoDaLXgND68bfVt,0.687,0.677,2,-4.172,1,0.0508,0.000658,0.000146,0.0564,0.546,75.052,4
2,0MsrWnxQZxPAcov7c74sSo,0.290,0.827,8,-3.451,1,0.0362,0.000659,0.000003,0.3010,0.450,167.086,4
3,18wCJcoFmXZ0jfrFhF6cYS,0.460,0.426,0,-7.286,0,0.2180,0.232000,0.075400,0.0847,0.432,88.802,4


In [ ]:
#This should be the output

#df = pd.DataFrame(master_track_audiofeatures)
#df
#no_duplicates = df.drop_duplicates(['Album URI']) *Change to find duplicates of track_uri
#no_duplicates
#no_duplicates.to_csv('Data/artist_album_uri.csv',sep='|',index=False)

In [ ]:
def audio_features(self, tracks=[]):
    if isinstance(tracks, str):
        trackid = self._get_id("track",tracks)